# Implementacja Gpt-2 Simple 

Pobranie repozytorium z Githuba

In [ ]:
!git clone https://github.com/drfinkus/gpt-2-simple

Cloning into 'gpt-2-simple'...
remote: Enumerating objects: 548, done.
remote: Total 548 (delta 0), reused 0 (delta 0), pack-reused 548
Receiving objects: 100% (548/548), 228.60 KiB | 435.00 KiB/s, done.
Resolving deltas: 100% (290/290), done.


Import niezbędnych bibliotek

In [ ]:
import os
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
os.chdir('gpt-2-simple')
%tensorflow_version 1.x
!pip install -r requirements.txt
import gpt_2_simple.gpt_2 as gpt2
from datetime import datetime
from google.colab import files
from time import time
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [ ]:
# !nvidia-smi

Pobranie odpowiedniego wyczunego modelu

In [ ]:
gpt2.download_gpt2(model_name="1558M")

Fetching checkpoint: 52.4Mit [00:00, 1.57Git/s]                                                     
Fetching encoder.json: 52.4Mit [00:00, 1.50Git/s]                                                   
Fetching hparams.json: 52.4Mit [00:00, 2.77Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.47Git [00:19, 75.6Mit/s]
Fetching model.ckpt.index: 52.4Mit [00:00, 2.79Git/s]                                               
Fetching model.ckpt.meta: 52.4Mit [00:00, 1.89Git/s]                                                
Fetching vocab.bpe: 52.4Mit [00:00, 2.27Git/s]                                                      


Wybór datasetu jako pliku TXT


In [ ]:
file_name = "merged_clean.txt"

In [ ]:
%ls

content/          models/
docs/             README.md
gpt_2_simple/     requirements.txt
LICENSE           setup.py
MANIFEST.in       Train_GPT_2_1_5b_Text_Generating_Model_w_GPU.ipynb
merged_clean.txt


In [ ]:
!head merged_clean.txt


Lovely Ilonka

There was once a king's son who told his father that he wished to marry.

'No, no!' said the king; 'you must not be in such a hurry. Wait till you
have done some great deed. My father did not let me marry till I had won
the golden sword you see me wear.'

The prince was much disappointed, but he never dreamed of disobeying his


Użycie funkcji finetune, aby dotrenować sieć, zmienić jej słownik na bardziej związany z naszym tematem generacji bajek.  Do funckji podajemy argumenty takie jak dataset, nazwa modelu, ilość kroków, możemy podać ewentualny checkpoint, rodzaj stosowanej optymalizacji, współczynnik prędkości uczenia, oraz wskaźniki  zapisu stanu sieci oraz informację na ilu kartach będziemy wykonywać trening

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              checkpoint_dir='checkpoint',
              model_name='1558M',
              steps=50000,
              restore_from='fresh',
              optimizer='adafactor',
              learning_rate=0.0001,
              run_name='run1',
              print_every=10,
              sample_every=500,
              save_every=1000,
              overwrite=True,
              multi_gpu=True, #dla wiełu kart graficznych
              )

n_ctx:  1024
Loading checkpoint models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:17<00:00, 17.90s/it]


dataset has 3246411 tokens
Training...


Następnie używajac funkcji generate generujemy tekst, określając  prefix (początek napodstawie którego siec rozopczyna generację), długość tekstu wygenerowanego,  temperaturę (czyli czy system ma być bardziej odtwórzczy czy twórczy), top_k odpowiada za ograniczenie zbioru słów do pewnej ilości najlepsyzch pod względem dopasowania słów.

In [ ]:
gpt2.generate(sess,
              prefix="King of Glasses",
              length=500,
              temperature=0.9,
              top_p=0.9,
              top_k = 10,
              nsamples=4,
              batch_size=4,
              truncate='<|endoftext|>'
              )

Monster has came to library,

and been sleeping in the books, so did the crow eat the cats;
and it had a special purpose for its meal, and set it to eat
them: it wanted to do away with them as thoroughly as it could,
while it could. But one or two did not get off; the birds had
resembled cat and crow, and were as cunning as cats themselves.

"Do you see that little boy?" said the eagle. "This bird is not a
cat, but a crow, which is a thief, and has done us a great deal of
harm. You can see the way it has taken the poor children."

But the crow was very great with a great distance, and the
nearer it got, the stronger it was; so it went up the gallery, and
up to the walls, and sat down at the feet of the cats.

"My dear little boy," said the crow, "where did you get to you
from?"

"From libraries," said the cat, "but you know better than I
that from places above, where books are more plentiful."

"Oh, but you know," said the crow, "there is no place
else but the library."

"And the best 

In [ ]:
# files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Wnioski z GPT2 
Implementacja sieci Gpt2-simple umożliwia wytrenowanie sieci do generacji tekstów raczej mniejszych niż te, do których ją przezaczyliśmy. Wyniki są całkiem zadowalające, natomiast są dużo słabsze w porównaniu do trenowania sieci o dużo wiekszej ilości parametrów takie jak GPT-3. 

Poniżej zamieszczamy porównanie  wyników GPT-2 Simple z półtora miliarda  wag oraz 50'000 kroków oraz 355 milionów wag oraz 6500 kroków. 

# Tekst 1 Półtora miliarda wag oraz 50'000 kroków:

"Why did she leave us here?" asked Grettir.

"Why did you run away, and run to me?" asked his sisters.

"Why didn't I stay, and look after you?" asked Grettir; and they
said, "We had a bad journey for you to go away, though you had brought
us news of three sisters, the three sisters who had never known a brother
or a sister."

"Did you only run away," said Grettir, though it was scarcely
necessary; "and if you only ran away, tell us at the same time how
we can bring you home again."


"We have been to the four corners of the world, and never left the
house of our grandmother's father; now, we will take him away from us."
Said Grettir to his sisters.

And when they heard what they were going to do they cried the more,
and said, "He must die if he shall not tell them the news."

"Yes; but you have only been to the four corners of the world," said Grett,
"and never let your grandfather come to you."

"There is one of us who can do it; we are going to the city."

"And he does not know the country we are in?" said Grettir
with a growl of pain.

"Yes; he is a common vagabond," said the two sisters, in the
same voice, and they had their sisters' children with them, and, as
they had the same speech, each told the other what they knew.

When the one came home she was in a state of great joy to find her
children well, and said to her, "What ails you, daughter?" And
Grettir answered, "That which I have received from the three
sisters has given me one thought which drives me to the city to-day."

#  355 milionów wag, 6500 kroków:


woman's heart
from her breast; and they took her in a boat and landed her with a very
fine coat and crown and all.

The young queen thought she was very silly, and was very unhappy, and had
to sleep on the ground. At last she became so fond of the young prince
that she went to him one morning and said to him: "Prince, I think we must
make ourselves pretty for you."

He was delighted, so he answered: "Oh, lovely one, why don't I make myself
pretty for you?"

She put her arms round his neck and kissed him and said:

"Don't fear, I'll be with you always."  And she was really brave, although she
said so to him all the time she was inside him.

When they all went home again they were very happy, for the new-found
princess did not get tired and could keep her promise to be with her when
she liked. And the young prince got an even bigger coat and crown and was
always with her, and was always happy and content. Then she changed her
form and became a handsome young man, and Prince Bocchi went to see her; he
stopped at her house and was very happy, and she was married.

A wedding was about to be held for all the brothers, as it was the twenty
thousandth anniversary of the marriage. And when they reached the house the
first thing that happened was that a boat came into the yard.
And when the boat came into the yard they were so close that the young
princess put her mouth on the prince's hand and made a kiss on his forehead.
Then she put her arm round his neck and kissed him and gave him a great
throng.

Then she sat down with them, and talked very much with them about
the good old times and the good old princess, and how much the whole world
had improved in twenty thousand years.  Then the young princes began to
sing and play, and the young girls danced and played. "Here we are, in
a great forest where there is no water and where rivers flow over the rocks
and run down, and there is not even a rock left."  The young prince said:

"I will go in search of the water and the people are not near my house."

And then they walked together toward their houses and it was all lovely.

After a while they began to go from house to house with the young prince,
talking together about the good old times.

Then he said to his brothers: "They say that if people were to live like
mice in a dark corner, the sun wouldn't rise, nor do all the things
that happen in life. But if we all lived like mice in a dark corner, there
is a place where nobody can get into without being searched. And what would
they do if they saw me, and the great old house?"

Then they all laughed together, and put their heads together, and said:

"Well, if we are to live in this house, let's all live together as mice in a
dark corner, I suppose?"

"No, let you have a separate place and take care of yourselves, and don't
make yourselves so ugly that any man will go in."

So they all gave him rooms he could walk in and eat and drink with in the
darkest corner.  And it was a wonderful place, but no one went in.
Then he was lonely and poor, and the people laughed at him and
said, "This place is more beautiful than any forest in the world."

There he lived with the rich and the poor--beautiful houses for walking
about, and he loved the rich and the poor.

He had a great heart, and loved the rich and the poor.

He was so kind to his brother sons, and to his father, and he said to
them: "Do not be too proud. Don't be lazy; do not be too selfish, but
make a little of every moment to the world. Look, let me come through here.
When I do this I'll be able to do something about this lonely place, and I
will tell everyone at once because it is so lonely."

Then his brothers told him to go straight ahead to the window, but he
was so nervous that he did not know when he was to come or go.  And he
just sat there waiting! and he got so anxious when he did not get anything
he wanted.

Suddenly he sat down in the very corner and took his breath, and thought:
"What might happen now I am waiting for?  There is the old woman who I want
to marry, and she has a son